In [1]:
import pandas as pd

In [2]:
nba = pd.read_csv('../data/NBA_Player_of_the_Week.csv', parse_dates = ['Date'])

In [3]:
nba.head()

,Active season,Player,Team,Conference,Date,Position,Height,Weight,Age,Draft Year,Seasons in league,Season,Season short,Real_value
0,0,Micheal Ray Richardson,New Jersey Nets,NaN,1985-04-14,PG,6-5,189,29,1978,6,1984-1985,1985,1.0
1,0,Derek Smith,Los Angeles Clippers,NaN,1985-04-07,SG,6-6,205,23,1982,2,1984-1985,1985,1.0
2,0,Calvin Natt,Denver Nuggets,NaN,1985-04-01,F,6-6,220,28,1979,5,1984-1985,1985,1.0
3,0,Kareem Abdul-Jabbar,Los Angeles Lakers,NaN,1985-03-24,C,7-2,225,37,1969,15,1984-1985,1985,1.0
4,0,Larry Bird,Boston Celtics,NaN,1985-03-17,SF,6-9,220,28,1978,5,1984-1985,1985,1.0


In [4]:
nba['Player'].nunique()

282

In [5]:
nba.groupby('Team').agg({'Season':'max' , 'Date':'count'}).sort_values('Date', ascending = False).head()

,Season,Date
Team,,
Los Angeles Lakers,2012-2013,71
San Antonio Spurs,2017-2018,61
Houston Rockets,2018-2019,60
Cleveland Cavaliers,2017-2018,59
Miami Heat,2017-2018,57


In [6]:
nba.groupby('Position').agg({'Season':'max' , 'Date':'count'}).sort_values('Date', ascending = False)

,Season,Date
Position,,
G,2018-2019,184
SG,2018-2019,173
C,2018-2019,164
PF,2018-2019,142
F,2018-2019,137
PG,2018-2019,122
SF,2017-2018,119
FC,2018-2019,84
GF,2018-2019,50


In [7]:
nba.head()

SyntaxError: invalid syntax (<ipython-input-7-ff865755e182>, line 1)

In [ ]:
## Data Source: https://data.world/datadavis/nba-salaries
salary = pd.read_excel('../data/NBA_Salary_1990-2017.xlsx')
salary = salary.drop(columns = 'Register Value')
salary.head()

In [ ]:
salary.loc[(salary['Season Start'] == 2017) & (salary['Player Name'] == 'Stephen Curry')]

In [ ]:
salary201819 = pd.read_csv('../data/NBA_Salary_2018-2019.csv')
salary201819['Player Name'] = salary201819['Player'].str.split(pat = '\\', expand = True)[0]
salary201819['Salary in $'] = salary201819['2018-19'].str.slice(start = 1)
salary201819 = (salary201819[['Player Name', 'Tm', 'Salary in $']]
                .rename(columns = {'Tm': 'Team'}))
salary201819['Season Start'] = 2018
salary201819['Season End'] = 2019
salary201819 = pd.merge(salary201819, salary[['Team', 'Full Team Name']].drop_duplicates(), on = 'Team', how = 'left')

In [ ]:
salary201819.head()

In [ ]:
salarydf = pd.concat([salary, salary201819], sort = True)

In [ ]:
salarydf.head()

In [ ]:
salarydf.shape[0]

In [ ]:
abb = pd.read_csv('../data/NBA_Teams_Abb.csv')
abb.head()

In [ ]:
salarydf_clean = pd.merge(salarydf, abb, left_on = 'Team', right_on = 'Abbreviation/Acronym', how = 'left')
salarydf_clean

In [ ]:
## Find the mispell abbreviation team name
misabb = salarydf_clean.loc[salarydf_clean['Abbreviation/Acronym'].isna(), ['Team', 'Full Team Name']].drop_duplicates()
misabb = pd.merge(misabb, abb, left_on = 'Full Team Name', right_on = 'Franchise', how = 'left')
misabb

In [ ]:
## After searching online, we know that the true meaning of 'CHO' is 'Charlotte Hornets'and 'BRK' is 'Brooklyn Nets'
abb.loc[(abb.Franchise == 'Brooklyn Nets') | (abb.Franchise == 'Charlotte Hornets')]

In [ ]:
## Change Abb Team Name with a dictionary
change_abb_dic = {'PHO' : 'PHX', 'NOH': 'NOP', 'NJN':'BKN', 'CHO':'CHA', 'BRK':'BKN'}
salarydf['Team'] = salarydf["Team"].replace(change_abb_dic)

In [ ]:
## left join again to find the true full team name
salary_new_df = pd.merge(salarydf, abb, left_on = 'Team', right_on = 'Abbreviation/Acronym', how = 'left')
salary_new_df = salary_new_df.drop(columns = ['Full Team Name', 'Abbreviation/Acronym'])
salary_new_df.head()